In [98]:
# Importar librerías Generales

import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

## Haga una exploración rápida de sus datos, para eso haga un resumen de su conjunto de datos.

In [99]:
# Lectura de datos
df = pd.read_csv('movies.csv')

In [100]:
# Encuentra los índices y los datos NaN en la columna 'budget'
indices_nan = df[df['budget'].isna()].index
datos_nan = df['budget'][indices_nan]

# Imprime los resultados
print("Índices con NaN en la columna 'budget':", indices_nan)
print("Datos NaN en la columna 'budget':", datos_nan)


Índices con NaN en la columna 'budget': Index([], dtype='int64')
Datos NaN en la columna 'budget': Series([], Name: budget, dtype: object)


In [101]:
# pre procesamiento de datos en el dataframe
df['id'] = df['id'].astype(str)
df['budget'] = pd.to_numeric(df['budget'], errors='coerce')
df['genres'] = df['genres'].str.split('|')
# df['homePage'] = df['homepage'].toString()
df['productionCompany'] = df['production_companies'].str.split('|')
df['productionCompanyCountry'] = df['production_countries'].str.split('|')
df['productionCountry'] = df['production_countries'].str.split('|')
df['revenue'] = pd.to_numeric(df['revenue'], errors='coerce')
df['runtime'] = pd.to_numeric(df['runtime'], errors='coerce')
df['video'] = df['video'].astype(bool)
df['director'] = df['director'].str.split('|')
df['actors'] = df['actors'].str.split('|')
df['actorsPopularity'] = df['actors_popularity'].str.split('|')


KeyError: 'production_companies'

In [ ]:
# Tabla descriptiva de los datos
column_data = pd.DataFrame({
    'Nombre de la columna': df.columns,
    'Tipo de dato': df.dtypes.values,
    'Ejemplo de valor': df.iloc[0].values,
    })

column_data

,Nombre de la columna,Tipo de dato,Ejemplo de valor
0,id,object,5
1,budget,float64,4000000.0
2,genres,object,"[Crime, Comedy]"
3,homePage,object,https://www.miramax.com/movie/four-rooms/
4,productionCompany,object,Miramax|A Band Apart
5,productionCompanyCountry,object,US|US
6,productionCountry,object,United States of America
7,revenue,int64,4257354
8,runtime,int64,98
9,video,object,FALSE


El dataset obtenido de 'movies.csv' presenta información sobre 1000 películas diferentes. Presenta 27 variables (columnas) de tipo y rango variado. 